In [2]:
import torch
import torch.nn as nn
import os

import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm
import numpy as np

from scipy.stats import spearmanr
import csv

In [2]:
def read_nodes(file_path):
    int_list = []
    with open(file_path, 'r') as csvfile:
        csv_reader = csv.reader(csvfile)
        for row in csv_reader:
            for item in row:
                try:
                    int_list.append(int(item))
                except ValueError:
                    print(f"Warning: '{item}' could not be converted to an integer and was skipped.")
    return int_list

def calculate_one(path):
    # score = torch.load(path, map_location=torch.device('cpu'))  # _test_0225_regroup
    score = torch.load(path, map_location=torch.device('cpu'))
    # score = torch.rand(5000, 500)
    print("score shape:", score.shape)

    nodes_str = []
    for i in range(50):
        nodes_str.append(f"./checkpoints/{i}/train_index.csv")

    full_nodes = [i for i in range(4656)]

    node_list = []
    for node_str in nodes_str:
        numbers = read_nodes(node_str)
        index = []
        for number in numbers:
            index.append(full_nodes.index(number))
        node_list.append(index)

    loss_list = torch.load("./result/gt.pt", map_location=torch.device('cpu')).detach()

    approx_output = []
    for i in range(len(nodes_str)):
        score_approx_0 = score[node_list[i], :]
        sum_0 = torch.sum(score_approx_0, axis=0)
        approx_output.append(sum_0)

    print(len(loss_list), loss_list[0].shape)
    print(len(approx_output), approx_output[0].shape)

    res = 0
    counter = 0
    for i in range(score.shape[1]):
        tmp = spearmanr(np.array([approx_output[k][i] for k in range(len(approx_output))]),
                        np.array([loss_list[k][i].numpy() for k in range(len(loss_list))])).statistic
        if np.isnan(tmp):
            print("Numerical issue")
            continue
        res += tmp
        counter += 1

    print(counter)

    return res/counter, loss_list, approx_output


In [14]:
path = "./result/score.pt"
print(calculate_one(path)[0])

score shape: torch.Size([4, 1])


IndexError: index 3155 is out of bounds for dimension 0 with size 4

In [15]:
score = torch.load('./result/score_GIP.pt', map_location=torch.device('cpu'))
# score = torch.rand(5000, 500)
print("score shape:", score.shape)
print(score)

score shape: torch.Size([4, 1])
tensor([[0.0619],
        [0.0660],
        [0.0732],
        [0.0473]])


In [16]:
score = torch.load('./result/score.pt', map_location=torch.device('cpu'))
# score = torch.rand(5000, 500)
print("score shape:", score.shape)
print(score)

score shape: torch.Size([4, 1])
tensor([[0.0314],
        [0.0454],
        [0.0459],
        [0.0352]])


In [8]:
x = np.array([0.0656, 0.0806, 0.0821, 0.708])

x * (0.0338 / 0.0656)

array([0.0338    , 0.04152866, 0.04230152, 0.36479268])

In [4]:
train_gradients = torch.load('train_grad_0.pt', map_location=torch.device('cpu'))
test_gradients = torch.load('test_grad_0.pt', map_location=torch.device('cpu'))

print(train_gradients.shape, test_gradients.shape)

torch.Size([4, 1536]) torch.Size([1, 1536])


In [ ]:
# Compute dot product for dimensions between l_dim to u_dim
l_dim = 0
u_dim = l_dim + 50257*768

print(f"dot-product: {1e-3 * train_gradients[:, l_dim:u_dim] @ test_gradients[:, l_dim:u_dim].T}")

dot-product: tensor([[0.0916],
        [0.1316],
        [0.1209],
        [0.1038]], grad_fn=<MmBackward0>)


In [13]:
manual_beta_grad = torch.load('manual_beta_grad.pt', map_location=torch.device('cpu'))
manual_gamma_grad = torch.load('manual_gamma_grad.pt', map_location=torch.device('cpu'))

print(manual_beta_grad.shape, manual_gamma_grad.shape)

# stack them
manual_grad = torch.cat((manual_beta_grad, manual_gamma_grad), dim=1)
# stack train and test gradients
grad = torch.cat((train_gradients, test_gradients))

torch.Size([5, 768]) torch.Size([5, 768])


In [14]:
print(manual_grad.shape)

print(grad.shape)

torch.Size([5, 1536])
torch.Size([5, 1536])


In [16]:
print(manual_grad[0, :10])
print(manual_grad[0, 768:778])

print(grad[0, :10])
print(grad[0, 768:778])

tensor([-0.0056,  0.0055, -0.0154, -0.0061,  0.0099, -0.0260,  0.0063,  0.0004,
        -0.0052, -0.0093])
tensor([-0.0016, -0.0043, -0.0140, -0.0010,  0.0020, -0.0064, -0.0177, -0.0010,
         0.0006, -0.0009])
tensor([ 0.0082,  0.0223,  0.0720,  0.0053, -0.0103,  0.0330,  0.0909,  0.0049,
        -0.0031,  0.0047], grad_fn=<SliceBackward0>)
tensor([ 0.0288, -0.0283,  0.0791,  0.0311, -0.0509,  0.1338, -0.0326, -0.0022,
         0.0268,  0.0477], grad_fn=<SliceBackward0>)
